In [5]:
import tensorflow as tf 
import os

In [11]:
basepath = os.path.join(os.getcwd(),'images')
trainpath = os.path.join(basepath,'train')
valpath = os.path.join(basepath,'val')

Langkah selanjutnya, kita akan menerapkan ImageDataGenerator untuk data latih dan data validasi. ImageDataGenerator merupakan sebuah fungsi yang sangat berguna untuk mempersiapkan data latih dan data validasi. Beberapa kemudahan yang disediakan ImageDataGenerator antara lain, preprocessing data, pelabelan sampel otomatis, dan augmentasi gambar.

Augmentasi gambar merupakan sebuah teknik yang dapat digunakan untuk memperbanyak data latih dengan cara menduplikasi gambar yang telah ada dengan menambahkan variasi tertentu. Anda akan mempelajari lebih dalam terkait teknik ini pada kelas “Belajar Pengembangan Machine Learning”. Anda juga dapat melihat detail mengenai augmentasi gambar menggunakan ImageDataGenerator pada tautan berikut. 

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest'
)

In [15]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

Selanjutnya, siapkan data latih dan validasi dari kumpulan data gambar yang di-load dalam memori melalui fungsi flow() berikut.

In [16]:
train_generator = train_datagen.flow_from_directory(
    trainpath, # direktori data latih
    target_size = (150,150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
    batch_size = 4,
    class_mode = 'binary' # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
)

Found 192 images belonging to 2 classes.


In [17]:
validation_generator = test_datagen.flow_from_directory(
    valpath,
    target_size = (150, 150),
    batch_size = 4,
    class_mode = 'binary'
)

Found 20 images belonging to 2 classes.


Setelah data siap, kita bisa membangun model Convolutional Neural Network (CNN). Pembuatan model CNN pada keras mirip dengan pembuatan model Multi Layer Perceptron (MLP) yang dibahas pada modul sebelumnya. Perbedaannya terdapat pada empat lapis layer konvolusi dan max pooling. 

Anda tentu masih ingat bahwa fungsi dari layer konvolusi adalah untuk mengekstraksi atribut pada gambar. Sedangkan layer max pooling berguna untuk mereduksi resolusi gambar sehingga proses pelatihan model lebih cepat. Nah, pada model CNN, proses klasifikasi gambar hanya berfokus pada atribut-atribut unik yang membedakan tiap kategori. Sehingga, teknik ini dinilai lebih optimal dibandingkan hanya menggunakan model MLP yang membedakan tiap kategori dengan melihat keseluruhan piksel-piksel pada gambar.

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

melihat summary dari arsitektur model yang telah kita buat.

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

model yang kita buat terdiri dari empat lapis Convolutional dan MaxPoling layer, sebuah flatten layer, serta dua buah dense layer. Ingatlah bahwa dense layer terakhir merupakan output layer. Pada kasus klasifikasi biner, output model merupakan angka tunggal antara 0 dan 1. Sehingga, kita set dense layer terakhir = 1. Sementara itu, kolom “Param #” berisi informasi mengenai jumlah parameter pada tiap layer.

Selanjutnya, kolom “Output Shape” berisi informasi ukuran output yang dihasilkan tiap layer. Jika diperhatikan, ukuran input gambar yang telah didefinisikan sebelumnya adalah sebesar (150, 150). Tapi pada convolutional layer pertama, setiap satu input gambar akan menghasilkan ukuran output (148, 148) sebanyak 32 gambar. Ukuran tersebut berkurang karena kita menggunakan filter dengan ukuran (3, 3) dengan jumlah filter sebanyak 32 filter. Sehingga, tiap satu input gambar akan menghasilkan 32 gambar baru dengan ukuran (148, 148). 

Kemudian, resolusi tiap gambar akan diperkecil dengan tetap mempertahankan informasi pada gambar menggunakan MaxPoling layer yang berukuran (2, 2). Hal ini  akan menghasilkan ukuran output gambar sebesar (74, 74). Nah, proses tersebut juga berlaku untuk Convolutional dan MaxPoling layer yang lain. 

Berikutnya, mari perhatikan flatten layer. Output dari MaxPoling layer terakhir yang terdiri dari 512 gambar dengan ukuran (7, 7) akan diubah ke dalam bentuk array 1D (tensor 1D). Hal ini  akan menghasilkan output berukuran (25088). 

Nah, output tersebut kemudian masuk ke dalam dense layer pertama yang memiliki 512 neuron. Sehingga, ia akan menghasilkan output dengan ukuran (512). Selanjutnya, output ini akan masuk pada dense layer kedua yang memiliki 1 neuron sehingga akan menghasilkan output dengan ukuran (1). Output dari layer terakhir inilah yang digunakan sebagai hasil akhir model untuk kasus klasifikasi biner.

tahap selanjutnya adalah melakukan compile model tersebut menggunakan fungsi compile(). Pada tahap ini, kita juga menentukan loss function serta optimizer yang akan digunakan. Loss function yang digunakan pada kasus klasifikasi biner adalah "binary_crossentropy". Selain itu, optimizer yang digunakan  pada kasus ini adalah "Adam optimizer". Adam optimizer dipilih karena mudah diterapkan, lebih efisien secara komputasi dan kebutuhan memori yang lebih kecil.

In [20]:
model.compile(loss = 'binary_crossentropy',
              optimizer = tf.optimizers.Adam(),
              metrics = ['accuracy'])

### latih model dengan model.fit 

In [22]:
model.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=20,
    validation_data= validation_generator,
    validation_steps = 5,
    verbose=2
)

Epoch 1/20


2023-03-26 23:32:39.382390: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-03-26 23:32:55.326653: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


25/25 - 16s - loss: 0.7676 - accuracy: 0.4100 - val_loss: 0.6929 - val_accuracy: 0.5000 - 16s/epoch - 657ms/step
Epoch 2/20
25/25 - 14s - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6907 - val_accuracy: 0.5000 - 14s/epoch - 555ms/step
Epoch 3/20
25/25 - 17s - loss: 0.6822 - accuracy: 0.5000 - val_loss: 0.6614 - val_accuracy: 0.5500 - 17s/epoch - 662ms/step
Epoch 4/20
25/25 - 16s - loss: 0.7575 - accuracy: 0.5400 - val_loss: 0.6887 - val_accuracy: 0.5000 - 16s/epoch - 650ms/step
Epoch 5/20
25/25 - 15s - loss: 0.6861 - accuracy: 0.5300 - val_loss: 0.6930 - val_accuracy: 0.5000 - 15s/epoch - 594ms/step
Epoch 6/20
25/25 - 15s - loss: 0.6946 - accuracy: 0.4800 - val_loss: 0.6925 - val_accuracy: 0.5000 - 15s/epoch - 597ms/step
Epoch 7/20
25/25 - 17s - loss: 0.6942 - accuracy: 0.4800 - val_loss: 0.6937 - val_accuracy: 0.5000 - 17s/epoch - 661ms/step
Epoch 8/20
25/25 - 14s - loss: 0.6996 - accuracy: 0.5300 - val_loss: 0.6906 - val_accuracy: 0.5000 - 14s/epoch - 577ms/step
Epoch 9/20
25/25 - 

In [23]:
model.save('messy_clean_rooms')

INFO:tensorflow:Assets written to: messy_clean_rooms/assets


INFO:tensorflow:Assets written to: messy_clean_rooms/assets
